In [ ]:
# Time each cell
%load_ext autotime

In [ ]:
import os
import threading
import time

from matplotlib import pyplot as plt
%matplotlib inline

import h5py
import pandas as pd
from glob import glob

from IPython.display import Image, display
import ipywidgets as widgets

import numpy as np
from astropy.io import fits
from astropy.nddata.utils import Cutout2D, PartialOverlapError, NoOverlapError
from astropy.stats import sigma_clipped_stats
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats, sigma_clip
from astropy.time import Time
from astropy.utils.console import ProgressBar
from tqdm import tqdm

from matplotlib.ticker import FormatStrFormatter
from copy import copy

from piaa.utils import helpers
from piaa.utils import images as img_utils
from piaa import exoplanets
from piaa.observation import Observation
from piaa.utils import pipeline
from pocs.utils.images import fits as fits_utils

palette = copy(plt.cm.inferno)
palette.set_over('w', 1.0)
palette.set_under('k', 1.0)
palette.set_bad('g', 1.0)

In [ ]:
os.environ['PGPASSWORD'] = 'pan0pt3&-t3ss'
data_dir = '/var/panoptes/images/fields'

In [ ]:
# sequence = 'PAN001/Wasp80/14d3bd/20180609T095151'
# sequence = 'PAN001/Hd189733/14d3bd/20180614T093015'
sequence = 'PAN001/Hd189733/14d3bd/20180807T071517'

In [ ]:
stamp_size = (14, 14)

In [ ]:
# Download FITS files
fits_blobs = helpers.get_observation_blobs(sequence)
len(fits_blobs)

In [ ]:
# Download all the FITS files from a bucket
fits_files = list()
if fits_blobs:
    with tqdm(len(fits_blobs), 'Downloading FITS files'.ljust(25)) as bar:
        for i, blob in enumerate(fits_blobs):
            fits_fn = helpers.unpack_blob(blob, save_dir=data_dir)
            fits_files.append(fits_fn)
            bar.update(i)

fits_files = fits_files
num_frames = len(fits_files)

In [ ]:
# Plate-solve all the images - safe to run again
solved_files = list()
with tqdm(
        len(fits_files), 
        'Solving files'.ljust(25)) as bar:
    for i, fn in enumerate(fits_files):
        try:
            fits_utils.get_solve_field(fn, timeout=90)
            solved_files.append(fn)
        except Exception:
            print("Can't solve file {}".format(fn))
            continue

solved_files = solved_files

In [ ]:
len(solved_files)

In [ ]:
wcs = WCS(solved_files[0])

In [ ]:
# Lookup point sources
# You need to set the env variable for the password for TESS catalog DB (ask Wilfred)
# os.environ['PGPASSWORD'] = 'sup3rs3cr3t'
point_sources = pipeline.lookup_point_sources( 
    solved_files, 
    wcs=wcs, 
    force_new=True
)

In [ ]:
len(point_sources)

In [ ]:
point_sources.head()

In [ ]:
arcsecs = (point_sources.d2d.values * u.degree).to(u.arcsec)

In [ ]:
plt.scatter(point_sources.x.values, arcsecs)
plt.xlabel('Y position')
plt.ylabel('Δ arcsec')

In [ ]:
arcsec_bins = np.histogram(arcsecs, bins=np.linspace(0, 30, 20))

In [ ]:
pixel_scale = 10.3

fig, ax1 = plt.subplots()

ax1.hist(arcsecs, bins=np.linspace(0, 25, 20))

# ax2 = ax1.twiny()
s2 = arcsec_bins[1] / pixel_scale

ax1_ticks = ax1.get_xticks()
# ax2_scale = ax1_ticks / pixel_scale

# ax2.set_xlabel("Δ pixels", fontsize=10)
# ax2.set_xticks(ax2_scale)
# ax2.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# ax2.grid(color='r', linestyle='--', alpha=0.25, linewidth=2)

ax1.set_xlabel("Δ arcsec", fontsize=10)
ax1.set_ylabel("Num of sources")

fig.suptitle('Tess Catalog Offset', y=1.05, fontsize=14)
# fig.tight_layout()

In [ ]:
point_sources.snr.hist(bins=np.arange(0,100, 5))
plt.ylabel('Num of sources')
plt.title('SNR')

In [ ]:
point_sources.vmag.hist(bins=np.linspace(7, 15, 20))
plt.ylabel('Num of sources')
plt.title('Vmags')

In [ ]:
snr_limit = 10

In [ ]:
# Create stamps
stamps_fn = pipeline.create_stamp_slices(
    sequence,
    solved_files,
    point_sources[point_sources.snr >= snr_limit],
    stamp_size=stamp_size
)

In [ ]:
stamps_fn

In [ ]:
sequence = 'PAN001_Wasp80_14d3bd_20180609T095151.hdf5'
# stamps_fn = '/var/panoptes/psc/PAN001_Hd189733_14d3bd_20180614T093015.hdf5'
# sequence = 'PAN001_Tres3_14d3bd_20180624T063045.hdf5'
stamps_fn = '/var/panoptes/psc/' + sequence

In [ ]:
stamps = h5py.File(stamps_fn)

In [ ]:
len(stamps)

In [ ]:
picid = '243921117' # Wasp 80
# picid = '256364928'
# picid = '243952829'
# picid = '248311256'
# picid = '116264089' # Tres-3

In [ ]:
picid = list(stamps.keys())[np.random.randint(0, len(stamps))]

In [ ]:
picid in stamps

In [ ]:
vary_series = pipeline.find_similar_stars(
    picid, 
    stamps,
    out_fn='/var/panoptes/psc/similar_{}_{}.csv'.format(sequence.replace('/','_'), picid)
)

In [ ]:
vary_series.head()

In [ ]:
num_refs = 100
camera_bias = 2048
stamp_size = (14, 14)

In [ ]:
stamp_collection = np.array([pipeline.get_psc(str(idx), stamps) - camera_bias 
                           for idx in vary_series.index[:num_refs]])
print("Stamp collection shape: {}".format(stamp_collection.shape))

In [ ]:
normalized_collection = np.array([pipeline.normalize(s) for s in stamp_collection])

In [ ]:
coeffs = pipeline.get_ideal_full_coeffs(normalized_collection)

In [ ]:
num_frames = np.array(stamps[picid]['data']).shape[0]

In [ ]:
ideal = pipeline.get_ideal_full_psc(
    stamp_collection, 
    coeffs[0]
).reshape(num_frames, -1)

In [ ]:
target = stamp_collection[0].reshape(num_frames, -1)

In [ ]:
target.shape

In [ ]:
lc = (target.sum(1) / ideal.sum(1))

In [ ]:
ani = helpers.animate_stamp(target.reshape(num_frames, stamp_size[0], stamp_size[1]))
ani

In [ ]:
image_times = np.array(stamps.attrs['image_times'])

In [ ]:
image_times = [Time(t0, format='mjd').to_datetime() for t0 in image_times]

In [ ]:
lc0 = pd.DataFrame(lc, index=image_times, columns=['rel_flux'])

In [ ]:
lc0.head()

In [ ]:
target.shape

In [ ]:
lc0.plot()
plt.title('Relative flux')

In [ ]:
lc0.std()

In [ ]:
aperture_size = 4

diff = list()
for picid in ProgressBar(list(stamps.keys())):
    try:
        psc = pipeline.get_psc(str(picid), stamps) - camera_bias
    except ValueError:
        continue
    
    if float(stamps[picid].attrs['vmag']) > 13:
        continue

    try:
        rgb_stamp_masks = helpers.get_rgb_masks(
            psc[0].reshape(stamp_size[0], stamp_size[1]), 
            force_new=True
        )
    except ValueError:
        continue
    
    for frame_idx in range(psc.shape[0]):
        d0 = psc[frame_idx].reshape(stamp_size[0], stamp_size[1])

        star_pos_x = np.array(stamps[picid]['original_position'])[frame_idx][0]
        star_pos_y = np.array(stamps[picid]['original_position'])[frame_idx][1]
        slice0 = helpers.get_stamp_slice(star_pos_x, star_pos_y, stamp_size=stamp_size)

        try:
            y_pos, x_pos = np.argwhere(d0 == d0.max())[0]
            aperture_position = (x_pos, y_pos)
        except IndexError:
            print("No star position: ", frame_idx, slice0, star_pos_x, star_pos_y)
            continue

        color_flux = dict()
        for color, mask in zip('rgb', rgb_stamp_masks):

            d1 = np.ma.array(d0, mask=~mask)

            try:
                d2 = Cutout2D(d1, aperture_position, aperture_size, mode='strict')
            except (PartialOverlapError, NoOverlapError) as e:
                continue
            except Exception as e:
                continue

            diff.append({
                'picid': picid,                
#                 'obstime': observation.stamps['image_times'][frame_idx],
                'color': color,
                'value': d2.data.sum(),
            })

In [ ]:
len(diff)

In [ ]:
diff

In [ ]:
try:
    lc = pd.DataFrame(diff).set_index('obstime')          

    csv_file = '/var/panoptes/images/lc/{}_diff.csv'.format(sequence.replace('/', '_'))
    print("Writing csv to {}".format(csv_file))
    lc.to_csv(csv_file)
except Exception as e:
    print("Problem creating CSV file: {}".format(e))        

In [ ]:
lc = pd.read_csv('/var/panoptes/images/lc/{}_diff.csv'.format(sequence.replace('/', '_'))).set_index(['picid', 'obstime'])

In [ ]:
lc.head()

In [ ]:
foo = lc.groupby(by=['picid', 'color']).describe()

In [ ]:
foo.to_csv('/var/panoptes/g_stats_{}_desc.csv'.format(sequence.replace('/', '_')))

In [ ]:
picid_list = lc.picid.unique()

stats = list()
for picid in tqdm_notebook(picid_list):
    rows = lc.picid == picid
    color = lc.color == 'g'

    l0 = lc[rows & color]
    
    count_mean, count_median, count_std = sigma_clipped_stats(l0.value)
    
    mag = -2.5 * np.log10(l0.value / 120)

    mag_mean, mag_median, mag_std = sigma_clipped_stats(mag)
    
#     std = (l0.value - l0.value.mean()).std()
#     avg_std = (std / l0.value).mean()

    # Get the vmag
    vmag = observation.stamps[str(picid)].attrs['vmag']
    
    stats.append([vmag, count_mean, count_median, count_std, mag_mean, mag_median, mag_std])

In [ ]:
g_stats = pd.DataFrame(noise, columns=['vmag','count_mean', 'count_median', 'count_std', 'mag_mean', 'mag_median', 'mag_std'], index=picid_list)

In [ ]:
g_stats.to_csv('/var/panoptes/g_stats_{}.csv'.format(sequence))

In [ ]:
plt.style.use('bmh')

In [ ]:
mags = g_stats.vmag
stds = g_stats.mag_std

In [ ]:
plt.figure(figsize=(12,9))
plt.scatter(mags, stds, alpha=0.25)
plt.xlim([6,14])
plt.ylim([0, .2])
plt.xlabel('V mag')
plt.ylabel('Standard dev. (mag.)')

In [ ]:
lc.groupby(by='picid').vmag.hist(bins=np.arange(6,17))

In [ ]:
lc.vmag.hist(bins=np.arange(6,17))